In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

import csv
import json

Руководители хотят получить в едином файле информацию о действующих полисах страхования, обогащённую дополнительными атрибутами:

инициалами страхователя (в виде Ф. И. О.);
полом;
возрастом;
суммой выплаты по убытку.

In [13]:
contracts_df = pd.read_excel('D:/DO_NOT_REMOVE/Data_Analyst_Junior/17/CASE_CONTRACTS.xlsx')
contracts_df.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,1096,2000000
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000


In [10]:
contracts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711 entries, 0 to 3710
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   contract_id       3711 non-null   int64 
 1   contract_num      3711 non-null   object
 2   product_name      3711 non-null   object
 3   client_id         3711 non-null   int64 
 4   contract_status   3711 non-null   object
 5   currency_name     3711 non-null   object
 6   duration          3711 non-null   int64 
 7   country           3711 non-null   object
 8   price             3711 non-null   int64 
 9   insurance_amount  3711 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 290.1+ KB


In [19]:
contracts_df['contract_status'].value_counts(dropna=False)

contract_status
Действует    3315
Завершен      396
Name: count, dtype: int64

In [20]:
# Отбор только действующих полисах из contracts_df
active_contracts_df = contracts_df[contracts_df['contract_status']=='Действует']
active_contracts_df.shape

(3315, 10)

In [14]:
clients_df = pd.read_excel('D:/DO_NOT_REMOVE/Data_Analyst_Junior/17/CASE_CLIENTS.xlsx')
clients_df.head()

,client_id,last_name,first_name,middle_name,age,sex
0,10000041307,З*****,Р*****,О*****,30,F
1,10000133158,И*****,М*****,Ю*****,37,M
2,100003773,Е*****,О*****,С*****,50,M
3,10000535718,Е*****,Ф*****,Ч*****,45,M
4,10000583119,И*****,Ф*****,Ф*****,28,M


In [15]:
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711 entries, 0 to 3710
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   client_id    3711 non-null   int64 
 1   last_name    3711 non-null   object
 2   first_name   3693 non-null   object
 3   middle_name  3453 non-null   object
 4   age          3711 non-null   int64 
 5   sex          3711 non-null   object
dtypes: int64(2), object(4)
memory usage: 174.1+ KB


In [21]:
# Объединение таблиц active_contracts_df и clients_df
active_contracts_with_names_df = active_contracts_df.merge(clients_df, on='client_id', how='left')
active_contracts_with_names_df.shape

(3315, 15)

In [16]:
losses_df = pd.read_excel('D:/DO_NOT_REMOVE/Data_Analyst_Junior/17/CASE_LOSSES.xlsx')
losses_df.head()

,loss_id,client_id,loss_name,loss_payout_amt
0,1,10000713193,Оказание медицинской помощи,100000
1,2,100014656,Оказание медицинской помощи,50000
2,3,10006545463,Оказание медицинской помощи,30000
3,4,10006611194,Оказание медицинской помощи,70000
4,5,1001091597,Оказание медицинской помощи,60000


In [17]:
losses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   loss_id          45 non-null     int64 
 1   client_id        45 non-null     int64 
 2   loss_name        45 non-null     object
 3   loss_payout_amt  45 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ KB


In [23]:
# Объединение таблиц active_contracts_with_names_df и losses_df
extended_df = active_contracts_with_names_df.merge(losses_df[['client_id','loss_payout_amt']], on='client_id', how='left')
extended_df.shape

(3315, 16)

In [24]:
extended_df.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount,last_name,first_name,middle_name,age,sex,loss_payout_amt
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,1096,2000000,А*****,Х*****,А*****,20,M,NaN
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000,З*****,Х*****,М*****,55,M,NaN
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000,Б*****,В*****,Щ*****,61,F,NaN
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000,И*****,У*****,Н*****,25,M,NaN
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000,З*****,Ч*****,Г*****,33,M,NaN


In [84]:
# Сохранить новый DataFrame в файл CSV
extended_df.to_csv('D:/DO_NOT_REMOVE/Data_Analyst_Junior/17/extended_df.csv', index=False) 

Все суммы (стоимость полиса price, страховую сумму insurance_amount, сумму выплаты по убытку loss_payout_amt) необходимо перевести в доллары США по последнему актуальному курсу (если страховая сумма значится в долларах США изначально, считаем, что курс актуальный) и добавить эту информацию в виде отдельных столбцов в витрину.



In [85]:
extended_df['currency_name'].value_counts(dropna=False)

currency_name
Российский рубль    3210
Доллар США           105
Name: count, dtype: int64

1. Получаем курс ЦБ на вчерашний день и делаем для него отдельный дф course_of_exchange_df

In [27]:
r = requests.get('https://cbr.ru/scripts/XML_daily.asp?date_req=10/07/2024')

In [29]:
with open('cbr.xml', 'w', encoding='utf-8') as f:
    f.write(r.text)

In [36]:
def export_xml_to_csv_func(encoding='utf-8'): 
    parser = ET.XMLParser(encoding=encoding)
    tree = ET.parse('cbr.xml', parser=parser)
    root = tree.getroot()

    with open('cbr.csv', 'w',newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for Valute in root.findall('Valute'):
            NumCode = Valute.find('NumCode').text
            CharCode = Valute.find('CharCode').text
            Nominal = Valute.find('Nominal').text
            Name = Valute.find('Name').text
            Value = Valute.find('Value').text
            writer.writerow([root.attrib['Date']] + [Valute.attrib['ID']] + [NumCode] + [CharCode] + [Nominal] +
                            [Name] + [Value.replace(',', '.')])

In [37]:
export_xml_to_csv_func(encoding='utf-8')

In [44]:
df = pd.read_csv('cbr.csv', header=None)
df.head()

,0,1,2,3,4,5,6
0,10.07.2024,R01010,36,AUD,1,Австралийский доллар,59.3493
1,10.07.2024,R01020A,944,AZN,1,Азербайджанский манат,51.7665
2,10.07.2024,R01035,826,GBP,1,Фунт стерлингов Соединенного королевства,112.9344
3,10.07.2024,R01060,51,AMD,100,Армянских драмов,22.6806
4,10.07.2024,R01090B,933,BYN,1,Белорусский рубль,27.4486


In [45]:
course_of_exchange_df = df.rename(columns={
    0: 'rate_dt', 
    2: 'currency_id', 
    3: 'currency_cd', 
    4: 'nominal_qty', 
    5: 'currency_name',
    6: 'currency_rate'})[['rate_dt', 'currency_id', 'currency_cd', 'nominal_qty','currency_name', 'currency_rate']]

course_of_exchange_df.head()

,rate_dt,currency_id,currency_cd,nominal_qty,currency_name,currency_rate
0,10.07.2024,36,AUD,1,Австралийский доллар,59.3493
1,10.07.2024,944,AZN,1,Азербайджанский манат,51.7665
2,10.07.2024,826,GBP,1,Фунт стерлингов Соединенного королевства,112.9344
3,10.07.2024,51,AMD,100,Армянских драмов,22.6806
4,10.07.2024,933,BYN,1,Белорусский рубль,27.4486


In [86]:
exch_rate = course_of_exchange_df[course_of_exchange_df['currency_cd']=='USD'].currency_rate.iloc[0]
exch_rate

88.0031

2. Дополняем extended_df колонками для цен в валюте и информацией из course_of_exchange_df

In [87]:
final_df = extended_df.copy()
final_df.shape

(3315, 16)

In [88]:
# Функция для конвертации значений
def convert_to_usd(row, column_name, exchange_rate):
    return round(row[column_name] * exchange_rate, 2) if row['currency_name'] == 'Российский рубль' else row[column_name]

In [89]:
# Применение функции к разным столбцам
columns_to_convert = ['price', 'insurance_amount', 'loss_payout_amt']
for column in columns_to_convert:
    final_df[f'{column}_USD'] = final_df.apply(convert_to_usd, column_name=column, exchange_rate=exch_rate, axis=1)

In [90]:
final_df.shape

(3315, 19)

In [93]:
final_df.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount,last_name,first_name,middle_name,age,sex,loss_payout_amt,price_USD,insurance_amount_USD,loss_payout_amt_USD
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,1096,2000000,А*****,Х*****,А*****,20,M,NaN,96451.40,176006200.0,NaN
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000,З*****,Х*****,М*****,55,M,NaN,168789.95,440015500.0,NaN
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000,Б*****,В*****,Щ*****,61,F,NaN,96451.40,176006200.0,NaN
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000,И*****,У*****,Н*****,25,M,NaN,134996.76,176006200.0,NaN
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000,З*****,Ч*****,Г*****,33,M,NaN,168789.95,440015500.0,NaN


In [94]:
#Удаляем дубликаты
final_df = final_df.drop_duplicates()
final_df.shape

(3315, 19)

In [96]:
# Сохранить новый DataFrame в файл xlsx
final_df.to_excel('D:/DO_NOT_REMOVE/Data_Analyst_Junior/17/final_df.xlsx', index=False)